In [ ]:
import itertools
import pandas as pd

def generate_lucky_numbers():
    """
    Generate lucky numbers based on specific criteria and target sums.
    
    This function generates lucky numbers by considering different combinations of four digits
    and checking their sums against predefined target sums. It also assigns categories to
    the lucky numbers based on certain conditions.
    
    Returns:
        pandas.DataFrame: A DataFrame containing generated lucky numbers and their categories.
    """
    # Define the target sums and possible digits
    target_sums = [8, 11, 13, 15, 16, 17, 18, 21, 23, 24, 25, 29, 31, 32, 33, 35, 37, 39]

    # Define lucky number categories and their associated values
    lucky_dict = {
        '伏位': [11, 22, 33, 44, 66, 77, 88, 99],
        '延年': [19, 91, 78, 87, 43, 34, 26, 62],
        '生氣': [14, 41, 67, 76, 93, 39, 28, 82],
        '天醫': [13, 31, 68, 86, 94, 49, 72, 27],
    }

    # Initialize a list to store lucky numbers
    lucky_numbers = []

    # Generate all possible combinations of four digits
    combinations = itertools.product(range(10), repeat=4)

    # Iterate through each combination
    for combination in combinations:
        if 4 not in combination:
            total_sum = sum(combination)

            # Check if the total sum matches a target sum
            if total_sum in target_sums:
                sorted_combination_int = ''.join(map(str, combination))

                # Determine the lucky number category
                category = "普通"
                for key, values in lucky_dict.items():
                    if int(sorted_combination_int) % 100 in values:
                        category = key
                        break
                # Append the lucky number to the list
                lucky_numbers.append({"類別": category, "號碼": sorted_combination_int, '總和': total_sum})
    
    # Create a DataFrame from the list of lucky numbers and drop duplicates
    lucky_number_df = pd.DataFrame(lucky_numbers).drop_duplicates().reset_index(drop=True)
    
    return lucky_number_df

# Generate lucky numbers and save them to an Excel file
lucky_numbers_df = generate_lucky_numbers()
lucky_numbers_df.to_excel("./result/lucky_number.xlsx", index=False)
lucky_numbers_df


In [ ]:
import re

section_list = [{"20":"臺北市區"}, {"40":"臺北區"}, {"50":"新竹區"}, {"60":"臺中區"}, {"70":"嘉義區"}, {"30":"高雄市區"}, {"80":"高雄區"}]
station_text = """["20-臺北市區監理所","21-士林監理站","25-基隆監理站","26-金門監理站","28-連江監理站"];
["40-臺北區監理所","41-板橋監理站","43-宜蘭監理站","44-花蓮監理站","45-玉里監理分站","46-蘆洲監理站"];
["50-新竹區監理所","51-新竹市監理站","52-桃園監理站","53-中壢監理站","54-苗栗監理站"];
["60-臺中區監理所","61-臺中市監理站","62-南投監理站埔里分站","63-豐原監理站","64-彰化監理站","65-南投監理站"];
["70-嘉義區監理所","71-東勢監理分站","72-雲林監理站","73-新營監理站","74-臺南監理站","75-麻豆監理站","76-嘉義市監理站"];
["30-高雄市區監理所","31-苓雅監理站","33-旗山監理站"];
["80-高雄區監理所","81-臺東監理站","82-屏東監理站","83-恆春監理分站","84-澎湖監理站"]"""

# Initialize a dictionary to store station information
station_dict = {}

# Regular expression pattern to match items inside []
pattern = r'\[([^]]+)\]'

# Loop through the split text and extract lists inside []
for idx, item in enumerate(station_text.split(';')):
    match = re.search(pattern, item)
    if match:
        extracted = eval(match.group(1))
        for i in extracted:
            station_code, station_name = i.split('-')
            section_code = list(section_list[idx].keys())[0]
            station_dict[station_name] = {"stationCode": station_code, "sectionCode": section_code}

import requests
import pandas as pd
from bs4 import BeautifulSoup

def extract_number_of_pages(url, headers, **params):
    """
    Extract the number of pages from the specified URL for a given set of parameters.
    
    Args:
        url (str): The URL pattern for querying auction data.
        headers (dict): HTTP headers for the request.
        **params: Additional parameters for the URL.

    Returns:
        int: The total number of pages with auction data.
    """
    response = requests.get(url.format(page=1, **params), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.find("span", id="pagebanner").get_text()
    numbers = [char for char in text.split('，')[0] if char.isdigit()]
    number = int(''.join(numbers))

    return number

def scrape_page_data(url, headers, page, action, **params):
    """
    Scrape data from a specific page of the Taiwan Motor Vehicle Driver Information Service (MVDis) website.
    
    This function sends a GET request to the provided URL with the specified page number and parameters.
    It then extracts auction or bidding data from the page's HTML content based on the action.
    
    Args:
        url (str): The URL pattern for querying the MVDis website.
        headers (dict): The headers for the HTTP request.
        page (int): The page number to scrape.
        action (str): The action to perform: 'bidding' or 'auction_records'.
        **params: Additional parameters to include in the URL for customization.

    Returns:
        list: A list of dictionaries containing scraped data for each row on the page.
    """
    print(page)
    response = requests.get(url.format(page=page, **params), headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr', class_=lambda x: x and ("even" in x or "odd" in x))
    page_data = []
    if action == 'bidding':
        for row in rows:
            cols = row.find_all('td')
            car_number = cols[0].text
            number = car_number[-4:]
            price = cols[3].text
            page_data.append({"car_number": car_number, "number": number, "price": price, "page": page})
    else:
        for row in rows:
            cols = row.find_all('td')
            car_number = cols[0].text
            number = car_number[-4:]
            price = cols[4].text
            page_data.append({"car_number": car_number, "number": number, "price": price, "page": page})

    return page_data
    
# 競標
def get_bidding_by_station(**params):
    """
    Scrape and analyze bidding data for a specific Taiwan Motor Vehicle Driver Information Service (MVDis) station.
    
    This function retrieves bidding data from the MVDis website for the specified station using the provided parameters.
    It then analyzes the car numbers and prices, calculates the sum of the last four digits of each car number,
    and determines if each car number is a lucky number based on a predefined lucky number DataFrame.
    The results are saved to an Excel file.
    
    Args:
        **params: Parameters including 'stationCode' for the station number.

    Returns:
        pd.DataFrame: DataFrame containing the scraped and analyzed bidding data.
    """
    action = 'bidding'
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
    }
    url = "https://www.mvdis.gov.tw/m3-emv-plate/bid/queryBiding?d-5481-p={page}&stationCode={stationCode}&onChangeItem=2&method=doChangeStation&sectionCode=4#gsc.tab=0"
    data_types = {'Column1': int, '類別': str, '號碼': str, '總和': str}
    lucky_number = pd.read_excel("./result/lucky_number.xlsx", dtype=data_types)
    
    output_data = []
    total_pages = extract_number_of_pages(url, headers, **params)
    
    for page in range(1, total_pages + 1):
        output_data.extend(scrape_page_data(url, headers, page, action, **params))
    
    output_df = pd.DataFrame(output_data)
    output_df = output_df.merge(lucky_number, how='left', left_on='number', right_on='號碼')
    output_df = output_df.sort_values(by='car_number')
    output_df.to_excel(f"./result/Bidding_{params['stationCode']}.xlsx", index=False)
    
    return output_df

# 競標紀錄
def get_auction_records_by_station(**params):
    """
    Scrape and analyze auction record data for a specific Taiwan Motor Vehicle Driver Information Service (MVDis) station.
    
    This function retrieves auction record data from the MVDis website for the specified station using the provided parameters.
    It then analyzes the car numbers and prices, calculates the sum of the last four digits of each car number,
    and determines if each car number is a lucky number based on a predefined lucky number DataFrame.
    The results are saved to an Excel file.
    
    Args:
        **params: Parameters including 'stationCode', 'queryYear', 'queryMonth', 'sectionCode', and 'queryDate'.

    Returns:
        pd.DataFrame: DataFrame containing the scraped and analyzed auction record data.
    """
    action = 'auction_records'
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36"
    }
    url = "https://www.mvdis.gov.tw/m3-emv-plate/bid/queryBid?queryType=2&CSRFToken=6fe10d2a-a0fb-479b-b650-2637d59e6a71&d-5481-p={page}&stationCode={stationCode}&onChangeItem=3&queryYear={queryYear}&queryMonth={queryMonth}&method=doChangeStation&sectionCode={sectionCode}&queryDate={queryDate}&queryBidType=2#gsc.tab=0"
    data_types = {'Column1': int, '類別': str, '號碼': str, '總和': str}
    lucky_number = pd.read_excel("./result/lucky_number.xlsx", dtype=data_types)

    output_data = []
    total_pages = extract_number_of_pages(url, headers, **params)
    
    for page in range(1, total_pages + 1):
        output_data.extend(scrape_page_data(url, headers, page, action, **params))
    
    output_df = pd.DataFrame(output_data)
    output_df = output_df.merge(lucky_number, how='left', left_on='number', right_on='號碼')
    output_df = output_df.sort_values(by='car_number')
    output_df.to_excel(f"./result/Auction_Records_{params['stationCode']}.xlsx", index=False)
    
    return output_df
    

In [ ]:
# Bidding Example usage
station_name = '宜蘭監理站'
station_info = station_dict.get(station_name, {})
station_code = station_info.get('stationCode')
params = {'stationCode': station_code}
result_df = get_bidding_by_station(**params)


In [ ]:
# Example usage
target_number = '1389'
filtered_df = result_df[result_df['number'] == target_number]
print(filtered_df)

In [ ]:
# Auction Records Example usage
station_name = '南投監理站'
station_info = station_dict.get(station_name, {})
queryYear=112
queryMonth=8
queryDate=16
params = {
    'stationCode': station_info.get('stationCode'), 
    'sectionCode': station_info.get('sectionCode'),
    'queryYear':queryYear,
    'queryMonth':queryMonth,
    'queryDate':queryDate,
    }
result_df = get_auction_records_by_station(**params)

In [ ]:
# Example usage
target_number = '1358'
filtered_df = result_df[result_df['number'] == target_number]
print(filtered_df)